### Reading Dataset

In [ ]:
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv("shampoo.csv")
data = data["Sales"]

### Dividing the Dataset into Training & Test Set

In [ ]:
size = int(len(data) * 0.80)
train, test = data[:size], data[size:]

### Plotting ACF & PACF

In [ ]:
import statsmodels.api as sm
sm.graphics.tsa.plot_acf(data, lags = 10)
sm.graphics.tsa.plot_pacf(data, lags = 10)

### Finding Optimal p, d, q

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np

import itertools
p = d = q = range(0, 5)
pdq_combinations = list(itertools.product(p, d, q))

min_rmse = 10 ** 9
optimal_pdq = (0, 0, 0)

for pdq in pdq_combinations:
    try: 
        model = ARIMA(train, order = pdq).fit()
        predict = model.forecast(len(test))
        error = np.sqrt(mean_squared_error(test, predict))
        if error < min_rmse:
            min_rmse = error
            optimal_pdq = pdq
    except:
        continue

print(optimal_pdq, min_rmse)

### Training the model with Train dataset

In [ ]:
model = ARIMA(train, order = optimal_pdq).fit()

In [ ]:
print(model.summary())

### Actual vs Predict

In [ ]:
predict = model.forecast(len(test))

plt.plot(train, label = "Training")
plt.plot(test, label = "Actual")
plt.plot(predict, label = "Forecast")
plt.title("Forecast vs Actual")
plt.legend()
plt.show()